--- Day 2: Red-Nosed Reports ---
Fortunately, the first location The Historians want to search isn't a long walk from the Chief Historian's office.

While the Red-Nosed Reindeer nuclear fusion/fission plant appears to contain no sign of the Chief Historian, the engineers there run up to you as soon as they see you. Apparently, they still talk about the time Rudolph was saved through molecular synthesis from a single electron.

They're quick to add that - since you're already here - they'd really appreciate your help analyzing some unusual data from the Red-Nosed reactor. You turn to check if The Historians are waiting for you, but they seem to have already divided into groups that are currently searching every corner of the facility. You offer to help with the unusual data.

The unusual data (your puzzle input) consists of many reports, one report per line. Each report is a list of numbers called levels that are separated by spaces. For example:

7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
This example data contains six reports each containing five levels.

The engineers are trying to figure out which reports are safe. The Red-Nosed reactor safety systems can only tolerate levels that are either gradually increasing or gradually decreasing. So, a report only counts as safe if both of the following are true:

The levels are either all increasing or all decreasing.
Any two adjacent levels differ by at least one and at most three.
In the example above, the reports can be found safe or unsafe by checking those rules:

7 6 4 2 1: Safe because the levels are all decreasing by 1 or 2.
1 2 7 8 9: Unsafe because 2 7 is an increase of 5.
9 7 6 2 1: Unsafe because 6 2 is a decrease of 4.
1 3 2 4 5: Unsafe because 1 3 is increasing but 3 2 is decreasing.
8 6 4 4 1: Unsafe because 4 4 is neither an increase or a decrease.
1 3 6 7 9: Safe because the levels are all increasing by 1, 2, or 3.
So, in this example, 2 reports are safe.

Analyze the unusual data from the engineers. How many reports are safe?

# 1

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.DataFrame()
with open('2.txt', 'r') as file:
    data["raw"] = [list(map(int, line.split())) for line in file]

data.head()

,raw
0,"[58, 59, 62, 63, 64, 63]"
1,"[71, 72, 74, 76, 78, 80, 82, 82]"
2,"[26, 29, 32, 34, 35, 39]"
3,"[9, 11, 14, 17, 19, 20, 21, 26]"
4,"[89, 92, 95, 93, 94, 97, 98]"


In [3]:
# check that we're dealing with integers
type(data["raw"][0][0])

int

In [4]:
data["monotonous?"] = data["raw"].apply(
    lambda lst: lst == sorted(lst) or lst == sorted(lst, reverse=True)
)

data["differ?"] = data["raw"].apply(
    lambda lst: all(1 <= abs(a - b) <= 3 for a, b in zip(lst, lst[1:]))
)

data["safe?"] = data["monotonous?"] & data["differ?"]
data

,raw,monotonous?,differ?,safe?
0,"[58, 59, 62, 63, 64, 63]",False,True,False
1,"[71, 72, 74, 76, 78, 80, 82, 82]",True,False,False
2,"[26, 29, 32, 34, 35, 39]",True,False,False
3,"[9, 11, 14, 17, 19, 20, 21, 26]",True,False,False
4,"[89, 92, 95, 93, 94, 97, 98]",False,True,False
...,...,...,...,...
995,"[20, 17, 14, 13, 12, 11, 8, 5]",True,True,True
996,"[9, 7, 6, 4, 2]",True,True,True
997,"[88, 90, 92, 93, 96, 99]",True,True,True
998,"[75, 76, 77, 78, 80, 83, 84, 86]",True,True,True


In [5]:
total = np.sum(data["safe?"])
total

402

--- Part Two ---
The engineers are surprised by the low number of safe reports until they realize they forgot to tell you about the Problem Dampener.

The Problem Dampener is a reactor-mounted module that lets the reactor safety systems tolerate a single bad level in what would otherwise be a safe report. It's like the bad level never happened!

Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.

More of the above example's reports are now safe:

7 6 4 2 1: Safe without removing any level.
1 2 7 8 9: Unsafe regardless of which level is removed.
9 7 6 2 1: Unsafe regardless of which level is removed.
1 3 2 4 5: Safe by removing the second level, 3.
8 6 4 4 1: Safe by removing the third level, 4.
1 3 6 7 9: Safe without removing any level.
Thanks to the Problem Dampener, 4 reports are actually safe!

Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports. How many reports are now safe?

# 2

In [9]:
data["safe_dampened?"] = data["raw"].apply(
    lambda lst: any( # for the same level-removed list ...
        ( # check for the dampened sort condition
            (lst[:i] + lst[i+1:] == sorted(lst[:i] + lst[i+1:]) or
             lst[:i] + lst[i+1:] == sorted(lst[:i] + lst[i+1:], reverse=True))
            and # check for the dampened distance condition
            all(1 <= abs(a - b) <= 3 for a, b in zip(lst[:i] + lst[i+1:], (lst[:i] + lst[i+1:])[1:]))
        )
        for i in range(len(lst))
    )
)
data

,raw,monotonous?,differ?,safe?,safe_dampened?
0,"[58, 59, 62, 63, 64, 63]",False,True,False,True
1,"[71, 72, 74, 76, 78, 80, 82, 82]",True,False,False,True
2,"[26, 29, 32, 34, 35, 39]",True,False,False,True
3,"[9, 11, 14, 17, 19, 20, 21, 26]",True,False,False,True
4,"[89, 92, 95, 93, 94, 97, 98]",False,True,False,True
...,...,...,...,...,...
995,"[20, 17, 14, 13, 12, 11, 8, 5]",True,True,True,True
996,"[9, 7, 6, 4, 2]",True,True,True,True
997,"[88, 90, 92, 93, 96, 99]",True,True,True,True
998,"[75, 76, 77, 78, 80, 83, 84, 86]",True,True,True,True


In [10]:
total = np.sum(data["safe_dampened?"])
total

455